
# Concept Drift Detection Benchmark Framework

**Objective**: A minimal framework for systematic evaluation of concept drift detection methods on real-world and synthetic datasets.

**Framework Components**:
- Real and synthetic dataset loading interface for standard concept drift benchmarks
- Unified drift detection algorithm interface
- Prequential evaluation methodology
- Statistical validation and result analysis



## Evaluation Methodology

### Classification Performance Metrics
- **Prequential Accuracy**: Classification accuracy using test-then-train evaluation protocol
- **Macro F1-Score**: Harmonic mean of precision and recall, macro-averaged across classes

### Drift Detection Performance Metrics
- **True Positive Rate**: Proportion of correctly detected concept drifts
- **False Alarm Rate**: Rate of incorrect drift detections per unit time
- **Detection Delay**: Average temporal delay between drift occurrence and detection


In [ ]:

# Enhanced Dependencies and Configuration

import math, random, time, warnings, sys, os
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Any, Union
from pathlib import Path
from collections import deque, defaultdict

# Core scientific computing libraries
import numpy as np
import pandas as pd

# Machine learning and evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import drift detection methods from methods directory
sys.path.insert(0, os.path.abspath('../methods'))
sys.path.insert(0, os.path.abspath('..'))

# Import River drift detectors
from river.drift import ADWIN
from river.drift.binary import DDM, EDDM, FHDDM, HDDM_A, HDDM_W
from river.datasets import synth

# Import custom methods (River-formatted)
from dawidd import DAWIDD
from shape_dd import ShapeDD
from new_d3 import D3

# Import validation pipeline
from validation_pipeline import ValidationPipeline, DriftDetectorEvaluator, DataStreamGenerator, ValidationMetrics
from visualization_utils import DriftVisualization

# Set style and seeds
plt.style.use('seaborn-v0_8')
np.random.seed(42)
random.seed(42)
warnings.filterwarnings('ignore')

# All modules imported successfully - validation pipeline ready


✅ All modules imported successfully!
✅ Comprehensive validation pipeline ready!
✅ Available methods: ADWIN, DDM, EDDM, HDDM_A, D3, ShapeDD, DAWIDD


## Step 1: Understanding Method Categories and Processing Approaches

The validation pipeline tests three distinct categories of drift detection methods, each with different computational approaches:

### Category 1: Streaming Optimized Methods (True Streaming)
- **ADWIN**: Adaptive windowing with statistical guarantees
- **DDM/EDDM/HDDM**: Binary error rate monitoring
- **Processing Approach**: One-point-at-a-time processing
- **Characteristics**: Minimal latency, designed for real-time applications
- **Best For**: Applications requiring immediate drift detection

### Category 2: Incremental Window-based Methods  
- **D3**: Discriminative drift detection with incremental learning
- **Processing Approach**: Optimized window-based processing with incremental updates
- **Characteristics**: Moderate computational cost, handles multivariate data
- **Best For**: Balanced performance between accuracy and computational efficiency

### Category 3: Batch Window-based Methods (Controlled Checking)
- **ShapeDD**: Shape analysis with Maximum Mean Discrepancy testing
- **DAWIDD**: Kernel-based independence testing  
- **Processing Approach**: Full window analysis with controlled checking frequency
- **Characteristics**: High computational cost, designed for maximum accuracy
- **Best For**: Offline analysis or applications where accuracy is more important than speed

## Step 2: Data Stream Generation

Before testing the methods, we need to generate appropriate data streams. The pipeline creates different types of synthetic data streams with known drift points to enable proper validation.

All methods follow the River interface and can be used interchangeably. The key difference is their computational approach and suitability for different data types and scenarios.


In [ ]:
# Step 2: Generate synthetic data streams with known drift points
generator = DataStreamGenerator()

# Generate different types of data streams for testing
univariate_stream, uni_drift_info = generator.generate_univariate_stream(length=1000, drift_points=[500])
multivariate_stream, mv_drift_info = generator.generate_multivariate_stream(length=800, n_features=3, drift_points=[300, 600])
binary_stream, bin_drift_info = generator.generate_binary_error_stream(length=1000, drift_points=[500])

# Legacy simple stream for backward compatibility
rng = random.Random(12345)
simple_data_stream = rng.choices([0, 1], k=1000) + rng.choices(range(4, 8), k=1000)

# Display stream characteristics
print(f"Generated streams:")
print(f"- Univariate: {len(univariate_stream)} points, drift at {[d.position for d in uni_drift_info]}")
print(f"- Multivariate: {len(multivariate_stream)} points, drift at {[d.position for d in mv_drift_info]}")
print(f"- Binary: {len(binary_stream)} points, drift at {[d.position for d in bin_drift_info]}")
print(f"- Legacy stream: {len(simple_data_stream)} points, drift at ~1000")


Quick Comparison of All Drift Detection Methods
Generated streams:
- Univariate: 1000 points, drift at [500]
- Multivariate: 800 points, drift at [300, 600]
- Binary: 1000 points, drift at [500]
- Simple legacy stream: 2000 points, drift at ~1000


## Step 3: Individual Method Testing

Let's test each method individually to understand their behavior. We'll start with streaming methods using appropriate data types.

In [ ]:
# Step 3a: Test ADWIN (streaming method for univariate data)
# ADWIN uses adaptive windowing - processes one point at a time
adwin = ADWIN(delta=0.002)
detections_adwin = []

for i, val in enumerate(univariate_stream):
    adwin.update(val)  # Process one data point
    if adwin.drift_detected:
        detections_adwin.append(i)
        print(f"ADWIN detected drift at position {i}")

# Calculate detection performance
true_drift_pos = [d.position for d in uni_drift_info]
detection_delays = [d - true_drift_pos[0] for d in detections_adwin if d > true_drift_pos[0]]
print(f"ADWIN: {len(detections_adwin)} detections at {detections_adwin}")
print(f"True drift at: {true_drift_pos}, Detection delay: {detection_delays}")


Testing ADWIN (Streaming Method)
------------------------------
ADWIN detected drift at position 511
ADWIN Summary: 1 detections at [511]
True drift at: [500]
Detection delay: [11]


### DDM

In [8]:
ddm = DDM()
print_warning = True
for i, x in enumerate(simple_data_stream):
    ddm.update(x)
    if ddm.warning_detected and print_warning:
        print(f"Warning detected at index {i}")
        print_warning = False
    if ddm.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True


Warning detected at index 47
Change detected at index 1006


ValueError: math domain error

### EDDM

In [9]:
eddm = EDDM()
print_warning = True
for i, x in enumerate(simple_data_stream):
    eddm.update(x)
    if eddm.warning_detected and print_warning:
        print(f"Warning detected at index {i}")
        print_warning = False
    if eddm.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True


Change detected at index 62
Warning detected at index 193
Change detected at index 294
Warning detected at index 360
Change detected at index 381
Warning detected at index 448
Change detected at index 454
Change detected at index 510
Change detected at index 596
Warning detected at index 647
Change detected at index 718
Change detected at index 774
Change detected at index 836
Warning detected at index 910
Change detected at index 922
Warning detected at index 986
Change detected at index 993


### HDDM-A

In [10]:
hddm_a = HDDM_A()
print_warning = True
for i, x in enumerate(simple_data_stream):
    hddm_a.update(x)
    if hddm_a.warning_detected and print_warning:
        print(f"Warning detected at index {i}")
        print_warning = False
    if hddm_a.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True


Warning detected at index 1004
Change detected at index 1005
Change detected at index 1009
Change detected at index 1041
Change detected at index 1070
Warning detected at index 1112
Change detected at index 1113
Change detected at index 1117
Change detected at index 1120
Change detected at index 1126
Change detected at index 1136
Warning detected at index 1146
Change detected at index 1147
Change detected at index 1152
Warning detected at index 1161
Change detected at index 1162
Change detected at index 1167
Change detected at index 1170
Change detected at index 1173
Change detected at index 1187
Warning detected at index 1191
Change detected at index 1192
Change detected at index 1196
Change detected at index 1214
Warning detected at index 1221
Change detected at index 1267
Warning detected at index 1273
Change detected at index 1274
Warning detected at index 1290
Change detected at index 1291
Warning detected at index 1299
Change detected at index 1301
Change detected at index 1306
C

### HDDM-W

In [11]:
hddm_w = HDDM_W()

print_warning = True
for i, x in enumerate(simple_data_stream):
    hddm_w.update(x)
    if hddm_w.warning_detected and print_warning:
        print(f"Warning detected at index {i}")
        print_warning = False
    if hddm_w.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True


Warning detected at index 314
Change detected at index 649
Change detected at index 1000
Warning detected at index 1040
Change detected at index 1042
Change detected at index 1049
Warning detected at index 1071
Change detected at index 1076
Warning detected at index 1128
Change detected at index 1145
Warning detected at index 1237
Change detected at index 1245
Warning detected at index 1290
Change detected at index 1291
Warning detected at index 1299
Change detected at index 1308
Warning detected at index 1335
Change detected at index 1342
Warning detected at index 1350
Change detected at index 1416
Warning detected at index 1466
Change detected at index 1502
Warning detected at index 1534
Change detected at index 1558
Warning detected at index 1591
Change detected at index 1612
Warning detected at index 1618
Change detected at index 1658
Warning detected at index 1704
Change detected at index 1722
Warning detected at index 1772
Change detected at index 1781
Warning detected at index 1

### FHDDMS

In [12]:
fhddm = FHDDM()
fhddm_s = FHDDM(short_window_size = 20)
for i, x in enumerate(simple_data_stream):
    fhddm.update(x)
    fhddm_s.update(x)
    if fhddm.drift_detected or fhddm_s.drift_detected:
        print(f"Change detected at index {i}")


Change detected at index 1038
Change detected at index 1235
Change detected at index 1257
Change detected at index 1385
Change detected at index 1523
Change detected at index 1539
Change detected at index 1653
Change detected at index 1763
Change detected at index 1770
Change detected at index 1977


### D3

In [13]:
from new_d3 import D3
d3 = D3()
for i, val in enumerate(simple_data_stream):
    # Convert single value to feature dict
    x = {'feature': float(val)}  # D3 expects dict format
    
    d3.update(x)
    
    if d3.drift_detected:
        print(f"D3 drift detected at index {i}, input value: {val}")


D3 drift detected at index 1099, input value: 7


### DAWIDD

In [14]:
# Create DAWIDD detector
from dawidd import DAWIDD
dawidd = DAWIDD(window_size=50, alpha=0.05)

# Update drift detector
for i, x in enumerate(simple_data_stream):
    dawidd.update(x)
    if dawidd.drift_detected:
        print(f"Change detected at index {i}")
        break


Change detected at index 49


### ShapeDD

In [15]:
from shape_dd import ShapeDD

detector = ShapeDD()

for i, x in enumerate(simple_data_stream):
    detector.update(x)
    if detector.drift_detected:
        print(f"Change detected at index {i}")
        break


Change detected at index 203


## Step 4: Comprehensive Validation Pipeline

Now we'll run the complete validation framework that systematically tests each method according to its specific algorithmic approach and computational characteristics.

### What This Step Does:
1. **Streaming Methods**: Tests with one-point-at-a-time processing on univariate and binary data
2. **Incremental Methods**: Tests with optimized window processing on multivariate data  
3. **Batch Methods**: Tests with controlled checking frequency to balance accuracy and computational cost
4. **Performance Metrics**: Calculates precision, recall, F1-score, detection delay, and processing time
5. **Visualization**: Generates comprehensive plots and comparison charts

In [ ]:
# Step 4: Run comprehensive validation pipeline
# This tests all methods according to their specific algorithmic approaches
pipeline = ValidationPipeline()
pipeline.run_comprehensive_validation()  # Generates metrics and visualizations



COMPREHENSIVE VALIDATION PIPELINE
Starting Comprehensive Drift Detection Validation Pipeline

1. Testing Univariate Streaming Detectors
----------------------------------------
Testing ADWIN...
  ADWIN Results:
    Precision: 1.000
    Recall: 1.000
    F1-Score: 1.000
    Avg Detection Delay: 23.0 points
    Avg Processing Time: 0.010 ms/point
    Total Detections: 3
    True Drift Points: [500, 1000, 1500]
    Detected Points: [511, 1023, 1535]


2. Testing Binary Error Detectors
----------------------------------------
Testing DDM...
  DDM Results:
    Precision: 0.000
    Recall: 0.000
    F1-Score: 0.000
    Avg Detection Delay: inf points
    Avg Processing Time: 0.002 ms/point
    Total Detections: 2
    True Drift Points: [600, 1200]
    Detected Points: [677, 1316]

Testing EDDM...
  EDDM Results:
    Precision: 0.133
    Recall: 1.000
    F1-Score: 0.235
    Avg Detection Delay: 2.0 points
    Avg Processing Time: 0.002 ms/point
    Total Detections: 15
    True Drift Points

## Step 5: Method Selection Guide

Based on the validation results, this guide helps you choose the most appropriate drift detection method for your specific use case and requirements.

In [ ]:
# Step 5: Method selection guide based on validation results
guide_data = {
    'Method': ['ADWIN', 'DDM/EDDM', 'D3', 'ShapeDD', 'DAWIDD'],
    'Data Type': ['Univariate', 'Binary/Error', 'Multivariate', 'Multivariate', 'Multivariate'],
    'Processing': ['Streaming', 'Streaming', 'Incremental', 'Batch', 'Batch'],
    'Best For': [
        'Real-time, guaranteed performance',
        'Classification error monitoring',
        'Moderate computational resources',
        'High accuracy requirements',
        'Statistical rigor, offline analysis'
    ],
    'Computational Cost': ['Low', 'Very Low', 'Medium', 'High', 'Very High'],
    'Latency': ['Very Low', 'Very Low', 'Low', 'Medium', 'High']
}

# Display method comparison table
df_guide = pd.DataFrame(guide_data)
print("Method Comparison:")
print(df_guide.to_string(index=False))

print("\nRecommendations by Use Case:")
print("Real-time applications: ADWIN, DDM")
print("Multivariate data analysis: D3, ShapeDD") 
print("High accuracy requirements: ShapeDD, DAWIDD")
print("Low latency requirements: ADWIN, DDM")
print("Research and offline analysis: DAWIDD, ShapeDD")
print("Limited computational budget: ADWIN, DDM")

print("\nProcessing Approaches:")
print("True Streaming (1-point-at-a-time): ADWIN, DDM, EDDM")
print("Incremental (optimized for streaming): D3")
print("Batch (controlled checking frequency): ShapeDD, DAWIDD")



DRIFT DETECTION METHOD SELECTION GUIDE
  Method    Data Type  Processing                            Best For Computational Cost  Latency
   ADWIN   Univariate   Streaming   Real-time, guaranteed performance                Low Very Low
DDM/EDDM Binary/Error   Streaming     Classification error monitoring           Very Low Very Low
      D3 Multivariate Incremental    Moderate computational resources             Medium      Low
 ShapeDD Multivariate       Batch          High accuracy requirements               High   Medium
  DAWIDD Multivariate       Batch Statistical rigor, offline analysis          Very High     High


Recommendations by Use Case:
------------------------------
🚀 Real-time applications: ADWIN, DDM
📊 Multivariate data: D3, ShapeDD
🎯 High accuracy needs: ShapeDD, DAWIDD
⚡ Low latency requirements: ADWIN, DDM
🔍 Research/offline analysis: DAWIDD, ShapeDD
💰 Limited computational budget: ADWIN, DDM


Streaming Characteristics:
-------------------------
✅ True Streaming (1

## Step 6: Custom Validation Function

This step provides a ready-to-use function for validating drift detectors on your own data. The function automatically detects your data type and applies appropriate validation strategies.


In [ ]:
# Custom Validation Function for Your Data
def validate_drift_detectors_on_custom_data(data_stream, true_drift_points, 
                                          detector_configs=None, 
                                          tolerance=50, 
                                          verbose=True):
    """
    Validate drift detectors on custom data.
    
    Parameters:
    -----------
    data_stream : list
        Your data stream (univariate, multivariate, or binary)
    true_drift_points : list
        Known drift point positions
    detector_configs : dict
        Configuration for each detector type
    tolerance : int
        Tolerance for detection delay
    verbose : bool
        Print detailed results
    
    Returns:
    --------
    dict : Validation results
    """
    if detector_configs is None:
        detector_configs = {
            'adwin': {'delta': 0.002},
            'd3': {'window_size': min(200, len(data_stream)//4), 'auc_threshold': 0.7},
            'shapedd': {'window_size': min(150, len(data_stream)//5), 'l1': 15, 'l2': 20, 'n_perm': 500},
            'dawidd': {'window_size': min(150, len(data_stream)//5), 'n_perm': 500}
        }
    
    results = {}
    
    # Determine data type
    sample = data_stream[0]
    is_multivariate = isinstance(sample, dict) or (hasattr(sample, '__len__') and len(sample) > 1)
    is_binary = all(isinstance(x, bool) or x in [0, 1] for x in data_stream[:10])
    
    if verbose:
        print(f"Data characteristics:")
        print(f"  Length: {len(data_stream)}")
        print(f"  Type: {'Multivariate' if is_multivariate else 'Binary' if is_binary else 'Univariate'}")
        print(f"  True drift points: {true_drift_points}")
        print()
    
    # Test appropriate detectors based on data type
    if not is_multivariate:
        # Test ADWIN
        adwin = ADWIN(**detector_configs['adwin'])
        detections = []
        for i, val in enumerate(data_stream):
            adwin.update(val)
            if adwin.drift_detected:
                detections.append(i)
        results['ADWIN'] = {
            'detections': detections,
            'precision': _calculate_precision(detections, true_drift_points, tolerance),
            'recall': _calculate_recall(detections, true_drift_points, tolerance)
        }
    
    if is_multivariate:
        # Test D3
        d3 = D3(**detector_configs['d3'])
        detections = []
        for i, sample in enumerate(data_stream):
            d3.update(sample)
            if d3.drift_detected:
                detections.append(i)
        results['D3'] = {
            'detections': detections,
            'precision': _calculate_precision(detections, true_drift_points, tolerance),
            'recall': _calculate_recall(detections, true_drift_points, tolerance)
        }
        
        # Test ShapeDD (with reduced frequency)
        shapedd = ShapeDD(**detector_configs['shapedd'])
        detections = []
        check_freq = max(10, len(data_stream) // 100)
        for i, sample in enumerate(data_stream):
            shapedd.update(sample)
            if i % check_freq == 0 and shapedd.drift_detected:
                detections.append(i)
        results['ShapeDD'] = {
            'detections': detections,
            'precision': _calculate_precision(detections, true_drift_points, tolerance),
            'recall': _calculate_recall(detections, true_drift_points, tolerance)
        }
        
        # Test DAWIDD (with reduced frequency)
        dawidd = DAWIDD(**detector_configs['dawidd'])
        detections = []
        for i, sample in enumerate(data_stream):
            dawidd.update(sample)
            if i % check_freq == 0 and dawidd.drift_detected:
                detections.append(i)
        results['DAWIDD'] = {
            'detections': detections,
            'precision': _calculate_precision(detections, true_drift_points, tolerance),
            'recall': _calculate_recall(detections, true_drift_points, tolerance)
        }
    
    if verbose:
        print("Validation Results:")
        print("-" * 50)
        for method, metrics in results.items():
            f1 = 2 * metrics['precision'] * metrics['recall'] / (metrics['precision'] + metrics['recall']) if (metrics['precision'] + metrics['recall']) > 0 else 0
            print(f"{method:10} | Precision: {metrics['precision']:.3f} | Recall: {metrics['recall']:.3f} | F1: {f1:.3f} | Detections: {metrics['detections']}")
    
    return results

def _calculate_precision(detections, true_drifts, tolerance):
    if not detections:
        return 0.0
    true_positives = 0
    for detection in detections:
        if any(abs(detection - true_drift) <= tolerance for true_drift in true_drifts):
            true_positives += 1
    return true_positives / len(detections)

def _calculate_recall(detections, true_drifts, tolerance):
    if not true_drifts:
        return 1.0
    detected_drifts = set()
    for detection in detections:
        for true_drift in true_drifts:
            if abs(detection - true_drift) <= tolerance:
                detected_drifts.add(true_drift)
                break
    return len(detected_drifts) / len(true_drifts)

# Custom validation function with automatic data type detection
print("Custom validation function ready!")
print("Usage: validate_drift_detectors_on_custom_data(your_data, your_drift_points)")


✅ Custom validation function ready!
Usage: validate_drift_detectors_on_custom_data(your_data, your_drift_points)


## Step 7: Example Usage and Validation

This final step demonstrates how to use the custom validation function with different types of data and provides templates for your own data validation.


In [ ]:
# Step 7: Example usage of custom validation function

# Example 1: Test on multivariate data
print("Example 1: Testing on generated multivariate data")
results = validate_drift_detectors_on_custom_data(
    multivariate_stream, 
    [d.position for d in mv_drift_info],
    tolerance=100  # Higher tolerance for batch methods
)

# Example 2: Test on univariate data  
print("\nExample 2: Testing on simple univariate data")
results_uni = validate_drift_detectors_on_custom_data(
    univariate_stream,
    [d.position for d in uni_drift_info],
    tolerance=50
)

# Template for user's own data
print("\nTemplate for your own data:")
print("# my_data = [your_data_here]  # List of numbers or dicts")
print("# my_drift_points = [300, 600]  # Known drift positions")
print("# results = validate_drift_detectors_on_custom_data(my_data, my_drift_points)")

print("\nValidation pipeline complete - check visualization files in experiments directory")


Example 1: Testing on generated multivariate data
Data characteristics:
  Length: 800
  Type: Multivariate
  True drift points: [300, 600]

Validation Results:
--------------------------------------------------
D3         | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Detections: [399, 699]
ShapeDD    | Precision: 0.000 | Recall: 0.000 | F1: 0.000 | Detections: []
DAWIDD     | Precision: 0.000 | Recall: 0.000 | F1: 0.000 | Detections: []

Example 2: Testing on simple univariate data
Data characteristics:
  Length: 1000
  Type: Univariate
  True drift points: [500]

Validation Results:
--------------------------------------------------
ADWIN      | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Detections: [511]

To use with your own data:
# Uncomment and modify the following:
# my_data = [your_data_here]  # List of numbers or dicts
# my_drift_points = [300, 600]  # Known drift positions
# results = validate_drift_detectors_on_custom_data(my_data, my_drift_points)

✅ Notebook successf